# Planet Painter
Planet Painter is able to download imagery from Planet.

In [1]:
# Standard Library Imports
import os
import glob

# Third-party Library Imports
import warnings

import geopandas as gpd

# Project-specific Imports
from coastseg_planet import processing
from coastseg_planet import download
from coastseg_planet.download import download_topobathy
from coastseg_planet import visualize
from coastseg_planet import model
from coastseg_planet import coregister
from coastseg_planet.masking import apply_cloudmask_to_dir
from coastseg_planet.utils import filter_files_by_area
from coastseg_planet import transects
from coastseg_planet import shoreline_extraction
from coastseg_planet.processing import create_elevation_mask_utm

from coastseg import file_utilities

warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')

c:\Users\sf230\anaconda3\envs\coastseg_planet\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


Transformers is only compatible with Keras 2, but you have explicitly set `TF_USE_LEGACY_KERAS` to `0`. This may result in unexpected behaviour or errors if Keras 3 objects are passed to Transformers models.


Version:  2.15.0
Eager mode:  True


# Inputs
1. Provide directory of downloaded Planet imagery
2. provide the location of the ROI used to download the Planet imagery
3. 

In [2]:
download_settings = {}

extract_shorelines_settings = {
    'output_epsg': 4326,
    'min_length_sl': 100,
    'dist_clouds': 50,
    'min_beach_area': 1000,
    'satname': 'planet',
}

coregister_settings = {
   'max_shift':2,
   'align_grids':True,
    "tieP_filter_level": 3,
    "min_reliability": 50,
    "grid_res": 100,  # tie point grid resolution in pixels of the target image (x-direction) (the lower this value the longer it takes)
   'v':True, #verbose mode
   'q':False, #quiet mode
    "r_b4match": 1,  # band of reference image to be used for matching (starts with 1; default: 1)
    "s_b4match": 1,  # band of source image to be used for matching (starts with 1; default: 1)
   'ignore_errors':False, # Recommended to set to True for batch processing
    'progress':True,
    'out_crea_options':["COMPRESS=LZW"], # otherwise if will use deflate which will not work well with our model
    'fmt_out':'GTIFF',
    'CPUs' : coregister.get_cpus()/2,
}
model_settings = {}

# Feature inputs
# ----------------
# ROI = Region of Interest
ROI_location = r"C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\roi.geojson"
transects_path = r"C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\transects.geojson"
sitename = 'AK'
# location of directory containing the downloaded imagery from Planet
planet_dir = ''
planet_dir = r"C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene"
good_dir = os.path.join(planet_dir, 'good')
# Optional Inputs
#----------------
# location of the file containing the reference landsat or planet image to coregister to
reference_path = ''
# location of the cloud mask of the reference landsat or planet image to coregister to
reference_bad_mask_path = ''

# Model Inputs
model_path = r"C:\development\coastseg-planet\CoastSeg-Planet\models\best_rgb.h5"
model_card_path = r'C:\development\coastseg-planet\CoastSeg-Planet\output_zoo\coastseg_planet\coastseg_planet\config\model_card.yml'
weights_directory = r'C:\development\doodleverse\coastseg\CoastSeg\src\coastseg\downloaded_models\segformer_RGB_4class_8190958'
model_card_path = file_utilities.find_file_by_regex(
    weights_directory, r".*modelcard\.json$"
)

Number of CPUs available: 20


## Controls

In [3]:
CONVERT_TOAR = False # If downloaded with TOAR tool was applied don't convert to TOAR again (set this to False)
RUN_GOOD_BAD_CLASSIFER = False  # Whether to run the classification model or not to sort the images into good and bad directories
APPLY_COREGISTER = False  # Whether to apply coregistration or not

## Filter the Files by Area

In [4]:
# Filter out the files that are less than 90% of the ROI area
filter_files_by_area(planet_dir,threshold=0.90,roi_path=ROI_location,verbose=True)

Moving 0 bad tifs to a new folder called bad in the directory: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene


# Convert the Planet Images to TOAR if not already TOAR
-todo use the fact the tifs generated by Toar clip in the name to set the value of CONVERT_TOAR

In [5]:
# convert the files in the directory to TOAR (Top of Atmosphere Reflectance) 
if CONVERT_TOAR:
    processing.convert_directory_to_TOAR(planet_dir,input_suffix='AnalyticMS_clip.tif',output_suffix='_TOAR.tif',separator='_3B')


# Convert Planet Imagery to Same Format as LandSat
- reorder to bands to be  RGBNIR
- scale the values to be 0 - 255 

In [6]:
# convert to RGB format that is compatible with the model. Range of values is 0-255
if CONVERT_TOAR:
    input_suffix='3B_TOAR.tif'
else:
    input_suffix='_3B_AnalyticMS_toar_clip.tif'
processing.convert_directory_to_model_format(planet_dir,input_suffix=input_suffix,output_suffix='_TOAR_model_format.tif',separator='_3B')

Converting files to model format: 100%|██████████| 91/91 [00:10<00:00,  8.70it/s]


## Filter the Planet Imagery
Use a model to sort the images as good or bad

In [7]:
# use a parameter to control if the good bad classification should run again
if RUN_GOOD_BAD_CLASSIFER:
    model.run_classification_model(model_path, planet_dir, planet_dir, regex= '*TOAR_model_format', move_files=False)
# if the classification model was not run then 
if not os.path.exists(good_dir):
    good_dir = planet_dir

# Coregister the Planet Imagery to the Reference Image
- We will coregister each modified planet image to the selected image that was modified to be in the zoo format

In [8]:
if APPLY_COREGISTER: 
    if not os.path.exists(reference_path):
        raise ValueError('Reference path to coregister all the imagery to does not exist. Please provide a reference path to coregister to.')
    if not os.path.exists(reference_bad_mask_path):
        raise ValueError('Reference bad mask path to coregister all the imagery to does not exist. Please provide a reference path to coregister to.')
    coregister.coregister_directory(good_dir,
                                    reference_path,
                                    reference_bad_mask_path,
                                    output_suffix='_TOAR_processed_coregistered_local.tif',
                                    bad_mask_suffix='_combined_mask.tif',
                                    use_local=True,  # if True, use local coregistration, otherwise use global coregistration
                                    overwrite=False, # if True, will overwrite the existing coregistered files, otherwise they will be skipped
                                    **coregister_settings)

# Apply the Model to the Planet Imagery

## Create an Elevation Mask from the Topobathy Data
- We will use this as our reference shoreline buffer
- This code is a bit of a placeholder eventually we will use the code to download the topobathy for the region 
- For now manually provide the topobathy tiff

In [9]:
from coastseg_planet.download import download_topobathy
import geopandas as gpd
# get the roi used to download the planet data
roi_gdf = gpd.read_file(ROI_location)
topobathy_tiff = download_topobathy(sitename,roi_gdf,planet_dir)

# Render the topobathy data on the map and have the user select a threshold to create a shoreline buffer
map_widget, get_low_threshold, get_high_threshold = visualize.create_interactive_map(topobathy_tiff)
map_widget

File already exists: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\AK_topobathy.tif


# Extract Shorelines
- Create a shoreline buffer in which to extract shorelines by setting a low and high threshold for the topobathy data. Shorelines will only be extracted within this region.

In [10]:
low_threshold = get_low_threshold()
high_threshold = get_high_threshold()
masked_elevation_tif_path = create_elevation_mask_utm(topobathy_tiff,low_threshold,high_threshold)
# get the output epsg from the roi
out_epsg = roi_gdf.estimate_utm_crs().to_epsg()
extract_shorelines_settings['output_epsg'] = out_epsg

# suffix of the tif files to extract shorelines from
separator = '_3B'
suffix = f"{separator}_TOAR_model_format"
if APPLY_COREGISTER:
    suffix = f"{separator}_TOAR_processed_coregistered_global"

filtered_tiffs = glob.glob(os.path.join(good_dir, f"*{suffix}.tif"))
if len(filtered_tiffs) == 0:
    print("No tiffs found in the directory")

# then intersect these shorelines with the transects
shorelines_dict = shoreline_extraction.extract_shorelines(good_dir,
                                                          suffix,
                                                          model_card_path,
                                                        extract_shorelines_settings,
                                                        masked_elevation_tif_path)

Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



N_DATA_BANDS: 3

saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211936_77_24c3_3B_TOAR_model_format_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211936_77_24c3_3B_TOAR_model_format_res.npz
32604
Saving figure to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\shoreline_detection_figures\2023-09-27-21-19-36_planet.jpg


Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211939_09_24c3_3B_TOAR_model_format_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211939_09_24c3_3B_TOAR_model_format_res.npz
32604
Saving figure to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\shoreline_detection_figures\2023-09-27-21-19-39_planet.jpg


Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231009_215918_25_2478_3B_TOAR_model_format_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231009_215918_25_2478_3B_TOAR_model_format_res.npz
32604
Saving figure to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\shoreline_detection_figures\2023-10-09-21-59-18_planet.jpg


Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231029_212230_99_2427_3B_TOAR_model_format_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231029_212230_99_2427_3B_TOAR_model_format_res.npz
32604
Saving figure to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\shoreline_detection_figures\2023-10-29-21-22-30_planet.jpg


Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231102_212052_38_24ce_3B_TOAR_model_format_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231102_212052_38_24ce_3B_TOAR_model_format_res.npz
32604
Saving figure to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\shoreline_detection_figures\2023-11-02-21-20-52_planet.jpg
saved extracted shorelines to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\raw_extracted_shorelines.geojson


# Intersect Extracted Shorelines with Transects (OPTIONAL)
- You will need to provide transects as a geojson file in CRS epsg 4326 in order to run this

In [11]:
# INTERSECT SHORELINES WITH TRANSECTS
if not os.path.exists(transects_path):
    raise ValueError('Transects path does not exist. Please provide a path to the transects to intersect with the shorelines.')
transects.intersect_transects(transects_path,shorelines_dict,extract_shorelines_settings['output_epsg'], save_location=good_dir)

Computing transect shoreline intersections: 100%|██████████| 9/9 [00:00<00:00, 214.88it/s]

Raw timeseries saved to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\raw_transect_time_series.csv
